# Data Quality Tutorial

Dataset: Homicidos, Colombia

In [ ]:
!pip install pylev

In [99]:
# Importing required libraries

import re
from random import randint
from datetime import datetime

import numpy as np
import pandas as pd

import pylev

In [100]:
# Parameter for showing all columns when printing a dataframe
pd.set_option('display.max_columns', None)

In [101]:
# Loading data
homicides_df = pd.read_csv("/data/homicides.csv")

In [102]:
# Printing the dataset dimensions
homicides_df.shape

(12400, 23)

In [ ]:
# Printing column data types
homicides_df.dtypes

In [104]:
# Deleting constant column
del homicides_df["2015"]

In [ ]:
homicides_df.head()

In [106]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [ ]:
# Applying the validation to all values in column
homicides_df["FECHA"].apply(dateparse)

# IT IS EXPECTED TO HAVE AN ERROR INTENTIONALLY

In [250]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [ ]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]

# THE LAST ERROR IS CAUSING THE ERROR

In [111]:
# Deleting an error by its index
homicides_df = homicides_df.drop([12399])

In [112]:
# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [113]:
# Counting homicides hour
homicides_df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

12    12399
Name: FECHA, dtype: int64

In [114]:
# The homicide hour is available in a different column!!!
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11]) + homicides_df["HORA"]

In [115]:
del homicides_df["HORA"]

In [ ]:
homicides_df["FECHA"].head()

In [117]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [118]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [ ]:
homicides_df.dtypes

In [ ]:
homicides_df.head()

In [121]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [122]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

<span style="color:red">TODO: Check and replace, if apply, the values identified as error because do not match with the dictionary</span>

In [142]:
#Se reemplazaron los dos valores encontrados como erróneos 'SAN ANDRÉS' y  'N. DE SANTANDER'
homicides_df = homicides_df.replace(['SAN ANDRÉS'], 'SAN ANDRÉS Y PROVIDENCIA')
homicides_df = homicides_df.replace(['N. DE SANTANDER'], 'NORTE DE SANTANDER')

In [143]:
#Se vuelve  a verificar los datos y obtendremos la lista vacía dado que no hay valores erróneos
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array([], dtype=object)

In [149]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep = False)]

In [148]:
duplicates_by_cedula.shape

(0, 21)

In [ ]:
# Showing some examples
duplicates_by_cedula.sort_values("CÉDULA", ascending = True).head(6)

<span style="color:red">TODO: Delete records with "CÉDULA" duplicated</span>

<span style="color:red">Hint: You can use drop_duplicates() function. Make sure to update original dataframe. Google it!</span>

In [ ]:
#Se estableció el parámetro subset con el valor de la cédula para eliminar los duplicados en esa columna
# y keep='first' para eliminar todas las filas excepto la primera  
homicides_df.drop_duplicates(subset=['CÉDULA'], keep='first')

In [166]:
#se verifica nuevamente el numero de registro 
homicides_df.shape

(11563, 21)

In [217]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].astype(str).apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [218]:
cedula_malformed.shape

(28, 21)

In [ ]:
cedula_malformed.head(6)

<span style="color:red">TODO: Fix the malformed "CÉDULA" values. Make sure to update original dataframe.</span>

In [227]:
data_1 = cedula_malformed.iloc[:1,:]

<span style="color:red">TODO: Make something similar to check and fix the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains allowed)</span>

In [ ]:
# Showing different values for "DIA" column
homicides_df["DIA"].unique()

In [ ]:
# Calculating the distance between two words using the Levenshtein method
pylev.levenshtein('sábado', 'sabaod')

In [ ]:
pylev.levenshtein('sábado', 'viernes')

<span style="color:red">TODO: Create a function to fix the digitation errors for column "DIA". Make sure to update original dataframe.</span>

In [ ]:
homicides_df["DIA"].unique()

In [ ]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].head(10)

<span style="color:red">TODO: Check and fix, if apply, the column "AÑO DE NACIMIENTO", using the column "EDAD". Make sure to update original dataframe.</span>